We need to install the leafmap client for data visualization in the notebook.

**Note:** This is set to run silently so you will not see an output when executing this cell. If you'd like to ensure the package downloads successfully remove the `--quiet` flag

![HLS Training](../images/HLS-inference.png)

In [7]:
! pip install ipyleaflet numpy --quiet

# Restart kernel, and hard refresh the browser

Import the python libraries required for running the script

In [8]:
import json
import ipyleaflet
import numpy as np

This is a dictionary contains several sample cases of burn scars across the globe. These configuration settings will automatically set the location of the map for AOI selection and the start_date and end_date to perform the data query.

In [9]:
#configure settings for selected events
INFERENCE_URL = <Update INFERENCE URL from credentials website>

EVENT_DETAILS = {
    'mongolian_fire': {
        'center_lon': 119.3,
        'center_lat': 47.1,
        'default_zoom': 8,
        'start_date': '2022-04-19T00:00:00Z',
        'end_date': '2022-04-19T23:59:59Z'
    },
    'new_mexico_black_fire': {
        'center_lon': -107.5,
        'center_lat': 33.5,
        'default_zoom': 10,
        'start_date': '2022-05-16T00:00:00Z',
        'end_date': '2022-06-10T23:59:59Z'
    },
    'alberta_fire': {
        'center_lon': -124.2,
        'center_lat': 61.8,
        'default_zoom': 8,
        'start_date': '2023-05-27T00:00:00Z',
        'end_date': '2023-05-28T23:59:59Z'
    },
    'maui_fire': {
        'center_lon': -156.659394,
        'center_lat': 20.886984,
        'default_zoom': 12,
        'start_date': '2023-08-13T00:00:00Z',
        'end_date': '2023-08-13T23:59:59Z'
    }
}

Select the predefined event of your choice from above. If you'd like to execute a different event, the simplest way to implement it would be to add the event details as a new key in the dictionary

In [10]:
event = 'maui_fire'
event_details = EVENT_DETAILS[event]

In [11]:
datestring = event_details['start_date']
HLSL30_TILE_LAYER = 'https://gitc-a.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?TIME=' + datestring + '&layer=HLS_L30_Nadir_BRDF_Adjusted_Reflectance&style=default&tilematrixset=GoogleMapsCompatible_Level12&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image%2Fpng&TileMatrix={z}&TileCol={x}&TileRow={y}'
HLSS30_TILE_LAYER = 'https://gitc-a.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?TIME=' + datestring + '&layer=HLS_S30_Nadir_BRDF_Adjusted_Reflectance&style=default&tilematrixset=GoogleMapsCompatible_Level12&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image%2Fpng&TileMatrix={z}&TileCol={x}&TileRow={y}'

Initialize map with one of the canned examples listed above, and HLS XYZ Layers from NASA Worldview. If you would like execute a custom use case, please navigate to your desired AOI, draw a bounding box, and update the start_date and end_date in the prepare_items method

In [21]:
from ipyleaflet import Map, TileLayer, DrawControl, GeoJSON

In [32]:
layer = TileLayer(url=HLSL30_TILE_LAYER, attribution='NASA',name='HLSL30', opacity=1)
draw_control = DrawControl()
map = Map(
        default_tiles=layer,
        center=(event_details['center_lat'],
        event_details['center_lon']), 
        zoom=event_details['default_zoom']
    )

draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.3
    }
}
hlsl30_tile_layer = TileLayer(url=HLSL30_TILE_LAYER, name='HLSL30', attribution='NASA')
hlss30_tile_layer = TileLayer(url=HLSS30_TILE_LAYER, name='HLSL30', attribution='NASA')
map.add_layer(hlsl30_tile_layer)
map.add_layer(hlss30_tile_layer)

map.add(draw_control)

# Store drawn shapes
drawn_shapes = []

# Define a function to handle drawing events
def handle_draw(self, action, geo_json):
    if action == 'created':
        drawn_shapes.append(geo_json)
        print("Shape added.")

# Attach the drawing event handler to the drawing control
draw_control.on_draw(handle_draw)
# map.add(layer)
map

Map(center=[20.886984, -156.659394], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

This cell saves the bounding box drawn by the user.

**WARNING:** if you skip the next step, the notebook will not find any HLS data as there will be no bounding box information.

**WARNING:** if you skipped the previous step, the notebook will not find any HLS data as there will be no bounding box information.

In [39]:
# Read the bounding box and print
bbox = drawn_shapes[0]

We will utilize the trained model, changed configuration file, and the date information to get a prediction on the selected region.

In [42]:

def bbox_from_geojson(bbox):
    """
        Get the coordinates of bounding box from an extended notation to flat coordinate
        notation
    Args:
        geojson: File path of geojson

    Returns:
        list: [left, down, right, top]
    """
    coordinates = np.asarray(bbox['geometry']['coordinates'])
    lats = coordinates[:, :, 1]
    lons = coordinates[:, :, 0]
    return [lons.min(), lats.min(), lons.max(), lats.max()]

In [43]:
# Convert geojson to flat bounding box representation.
bbox = bbox_from_geojson(bbox)
bbox

[-156.633883, 20.83925, -156.633883, 20.83925]

Note: `BUCKET_NAME`, and `identifier` are variables set in the previous notebook. Please copy paste those variables here for this step to run smoothly.


In [ ]:
BUCKET_NAME = <Update>
identifier = <Update> 

config_filename = 'configs/burn_scars_Prithvi_100M.py'
new_config_filename = f"configs/{identifier}-burn_scars_Prithvi_100M.py"
MODEL_NAME = f"{identifier}-workshop.pth"

In [ ]:
import requests
import json

# prepare payload

payload = json.dumps({
  "config_path": f"s3://{BUCKET_NAME}/data/{new_config_filename}",
  "model_path": f"s3://{BUCKET_NAME}/{MODEL_NAME}",
  "model_type": "burn_scars",
  "date": event_details['start_date'].split('T')[0],
  "bounding_box": bbox
})

headers = {
  'Content-Type': 'application/json'
}

# Use deployed app to get inference on the selected date/location
response = requests.request(
      "POST", 
      f"{INFERENCE_URL}/infer", 
      headers=headers, 
      data=payload
  )

predictions = response.json()

predictions

In [ ]:
geojson = predictions['predictions']

detection_map = Map(
        center=(event_details['center_lat'],
        event_details['center_lon']), 
        zoom=event_details['default_zoom'], 
    )
detection_map.add(hlsl30_tile_layer)
detection_map.add(hlss30_tile_layer)
detection_map.add(GeoJSON(data=geojson))

detection_map